<a href="https://colab.research.google.com/github/juoklee/ai-exploration-notebooks/blob/main/7_2_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet openai duckpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00


In [3]:
#open ai key
import os, openai
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('openai')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# 이전 방식
client = openai.OpenAI()

content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.
문장: \"나는 노란 망고가 좋더라\n
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
          "role": "user",
          "content": content
        }
    ],
)

print(response.choices[0].message.content)


{
  "색깔": "노란"
}


In [5]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema",
    "type": "object",
    "properties": {
        "color": {"type": "string"}
    },
    "required": ["color"]
}


content = f"""다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.
문장: '나는 노란 망고가 좋더라'
json schema 는 다음과 같다 : {schema}
색깔 정보가 없으면 no color로 대신해라"""


# 이전 방식
response = client.chat.completions.create(
   model="gpt-3.5-turbo-1106",
    messages=[
        {
          "role": "user",
          "content": content
        }
    ],
)

print(response.choices[0].message.content)

{
  "color": "yellow"
}


잘못된 출력 예
1. 틀린 아웃풋 (output 텍스트가 포함됨) : output: ~~
2. 사용자는 노란색을 좋아합니다.
3. 주신 내용에는 색깔 정보가 없는데, 다른 도와 드릴 어쩌구~?

In [7]:
# NEW!!!
response = client.chat.completions.create(
   model="gpt-4",
    messages=[
        {
          "role": "user",
          "content": "나는 노란색이 좋더라"
        }
    ],
   functions=[
       {
           "name": "color_extractor",
           "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력",
           "parameters": {
               "type": "object",
               "properties": {
                   "color": {
                       "type": "string",
                       "description": "사용자의 선호 색깔을 알아와서 영어 단어로 출력"
                   }
               },
               "required": ["color"],
           },
       }
   ]
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "color": "노란색"\n}', name='color_extractor'), tool_calls=None)


In [8]:
print(response.choices[0].message.function_call.arguments)

{
  "color": "노란색"
}


In [10]:
import os, openai

response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user", "content" : "퇴근했는데 일 시키는 사람"}],
    functions=[
        {
            "name" : "color_extractor",
            "description" : "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력",
            "parameters" : {
                "type":"object",
                "properties" : {
                    "color" : {
                        "type" : "string",
                        "description" : "사용자의 선호 색깔을 알아와서 영어 단어로 출력"
                    },
                },
                "required" : ["color"],
            },
        }
    ]

)

print(response.choices[0].message)

ChatCompletionMessage(content='힘든 상황이시군요. 차분하게 상황을 설명하고 잠시 쉬어야한다는 것을 어떤 이유에서든 간에 이야기해보는 것은 어떨까요? 여유 시간이 필요하다는 건 전혀 잘못된 건 아닙니다. 휴식은 생산성을 높이는 중요한 요소입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [11]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user", "content" : "아침에 세 개 저녁에 네 개"}],
    functions=[
        {
            "name" : "mango_add",
            "description" : "망고 숫자를 더함",
            "parameters" : {
                "type":"object",
                "properties" : {
                    "quantity1" : {
                        "type" : "integer",
                        "description" : "첫 번째 망고 숫자",
                    },
                    "quantity2" : {
                        "type" : "integer",
                        "description" : "두 번째 망고 숫자",
                    },
                },
                "required" : ["quantity1", "quantity2"],
            },
        }
    ],
    function_call="auto"
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "quantity1": 3, \n  "quantity2": 4\n}', name='mango_add'), tool_calls=None)


In [12]:
print(response.choices[0].message.function_call)

FunctionCall(arguments='{\n  "quantity1": 3, \n  "quantity2": 4\n}', name='mango_add')


In [14]:
import openai
import json
from duckpy import Client
import ast

duckduckgo_client = Client()

def duck_search(query) -> str:
  """Runs a duckduckgo search"""
  output = duckduckgo_client.search(query)
  return str(output)

completion = client.chat.completions.create(
    model="gpt-4",
    temperature=0,
    functions=[
        {
            "name": "duck_search",
            "description": "Runs a duckduckgo search",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Translate the Korean content into English input query",
                    },
                },
                "required": ["query"],
            },
        }
    ],
    messages=[
        {"role": "user", "content": "You must use the 'duck_search' function to get information."},
        {"role": "user", "content": "션 멘데스 나이가 몇이야?"},
    ]
)

In [17]:
print(completion.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "query": "션 멘데스 나이"\n}', name='duck_search'), tool_calls=None)


In [22]:
example_user_input = "런던 날씨가 안 좋다면서? 거기 날씨는 어때?"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user", "content": example_user_input}],
    functions=[
        {
            "name": "get_different_response",
            "description": "답변을 여러가지 말투로 하기",
            "parameters": {
                "type": "object",
                "properties": {
                    "response_a": {
                        "type": "string",
                        "description": "답변을 높임말로 바꿈",
                    },
                    "response_b": {
                        "type": "string",
                        "description": "답변을 반말로 바꿈",
                    },
                },
                "required": ["response_a", "response_b"],
            }
        }
    ],
    function_call={"name" : "get_different_response"},
)


reply_content = completion.choices[0].message
print(reply_content.function_call.arguments)

{
  "response_a": "네, 현재 런던은 비가 내리고 있어요. 온도는 8도로 조금 추워요. 따뜻하게 입고 다니시는 게 좋을 것 같아요.",
  "response_b": "어, 지금 런던은 비가 오고 있어. 온도는 8도로 쪼금 추워. 따뜻하게 입고 다녀야 할 거 같아."
}
